<a href="https://colab.research.google.com/github/nv-hiep/flight_delay_prediction/blob/master/step3_gradient_boosting_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 3: Gradient Boosting Model**

**Connect and authorize google drive with google colab:**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# !ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 1. Import Libraries



In [ ]:
import os
import numpy   as np
import pandas  as pd
import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline  

In [ ]:
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE, RFECV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, roc_curve, classification_report

from sklearn.neighbors import KNeighborsClassifier
from collections import defaultdict

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [ ]:
from hyperopt import fmin, hp, tpe, Trials, space_eval
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample

from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from functools import partial

from sklearn.svm import SVC

from sklearn import datasets
from sklearn.utils import shuffle

# 2. Data directory

In [ ]:
data_dir = '/content/gdrive/My Drive/data'
%cd '/content/gdrive/My Drive/data'

current_dir = os.getcwd()
print(current_dir)
data_path = os.path.join(data_dir, 'flights', '')
print(data_path)

/content/gdrive/My Drive/data
/content/gdrive/My Drive/data
/content/gdrive/My Drive/data/flights/


# 3. Read data

In [ ]:
# Read clean sub-data from csv
df_sub = pd.read_csv( os.path.join(data_path, 'cleaned_data_jan_20klines.csv') )
df_sub.tail()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,DEST,DEP_TIME,TAXI_OUT,TAXI_IN,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DELAYED
19995,8,7,11,695,5175,261,180,16.73,17.0,5.0,1855.0,11.0,0.0,0.0,0.0,84.0,3
19996,8,7,11,3050,4738,261,188,9.48,31.0,5.0,1845.0,0.0,0.0,0.0,0.0,0.0,1
19997,8,7,11,4131,2498,261,223,13.98,16.0,4.0,651.0,0.0,0.0,0.0,0.0,38.0,2
19998,8,7,11,3754,3226,261,223,20.70,14.0,4.0,651.0,0.0,0.0,0.0,0.0,136.0,4
19999,8,7,11,743,4798,261,223,8.35,17.0,4.0,651.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
# Checking the null values
df_sub.isnull().sum()

DAY_OF_MONTH           0
DAY_OF_WEEK            0
OP_UNIQUE_CARRIER      0
TAIL_NUM               0
OP_CARRIER_FL_NUM      0
ORIGIN                 0
DEST                   0
DEP_TIME               0
TAXI_OUT               0
TAXI_IN                0
DISTANCE               0
CARRIER_DELAY          0
WEATHER_DELAY          0
NAS_DELAY              0
SECURITY_DELAY         0
LATE_AIRCRAFT_DELAY    0
DELAYED                0
dtype: int64

In [ ]:
df_sub.dtypes

DAY_OF_MONTH             int64
DAY_OF_WEEK              int64
OP_UNIQUE_CARRIER        int64
TAIL_NUM                 int64
OP_CARRIER_FL_NUM        int64
ORIGIN                   int64
DEST                     int64
DEP_TIME               float64
TAXI_OUT               float64
TAXI_IN                float64
DISTANCE               float64
CARRIER_DELAY          float64
WEATHER_DELAY          float64
NAS_DELAY              float64
SECURITY_DELAY         float64
LATE_AIRCRAFT_DELAY    float64
DELAYED                  int64
dtype: object

In [ ]:
df_sub.columns

Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'TAIL_NUM',
       'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'DEP_TIME', 'TAXI_OUT',
       'TAXI_IN', 'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY',
       'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'DELAYED'],
      dtype='object')

In [ ]:
df_sub.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,DEST,DEP_TIME,TAXI_OUT,TAXI_IN,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DELAYED
0,16,1,0,1373,2186,76,234,9.22,26.0,5.0,1126.0,0.0,0.0,0.0,0.0,0.0,1
1,17,2,0,1258,2186,76,234,9.07,31.0,4.0,1126.0,0.0,0.0,0.0,0.0,0.0,0
2,18,3,0,1252,2186,76,234,9.13,44.0,5.0,1126.0,0.0,0.0,0.0,0.0,0.0,1
3,19,4,0,1477,2186,76,234,9.08,41.0,8.0,1126.0,0.0,0.0,29.0,0.0,0.0,1
4,20,5,0,1505,2186,76,234,9.05,20.0,6.0,1126.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
# Create data and label sets
y = df_sub['DELAYED'][:3000]
X = df_sub.drop(['DELAYED'], axis=1, inplace=False)[:3000]

In [ ]:
print(X.shape)
print(y.shape)

(3000, 16)
(3000,)


In [ ]:
# Training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 4. Gradien Boosting model

In [ ]:
# Get score of the GradientBoostingClassifier on the test data
def get_score_gradient_boosting(X_train, X_test, y_train, y_test):
    # Train the model
    clf = GradientBoostingClassifier(n_estimators=100, random_state=0)
    clf.fit(X_train, y_train)

    # Predict and get score on test data
    y_pred = clf.predict(X_test)
    score  = accuracy_score(y_test, y_pred)
    print('Accuracy: ', score)

    return score

## 4.1 Gradient Boosting model and Recursive Feature Elimination (RFE) 

In [ ]:
# Number of features in X_train
nfeatures = len( X_train.columns )
print('Number of features in X_train: ', nfeatures)

Number of features in X_train:  16


In [ ]:
estimator = GradientBoostingClassifier(n_estimators=100, random_state=0)
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(X, y)
print( 'Supports/features selected [True/False]: ', selector.support_ )
print( 'Ranking of the features: ', selector.ranking_ )

Supports/features selected [True/False]:  [False False False False False False False False False False False  True
 False  True False  True]
Ranking of the features:  [10 12 14  8  6 11  9  3  2  7  5  1  4  1 13  1]


In [ ]:
print('Number of selected features: ', selector.n_features_)
print()
print('Grid scores: ', selector.grid_scores_)
print()
print('Feature importances: ', selector.estimator_.feature_importances_ )

Number of selected features:  3

Grid scores:  [0.645      0.72333333 0.769      0.65666667 0.69633333 0.671
 0.67533333 0.68433333 0.69233333 0.69533333 0.703      0.70466667
 0.69533333 0.697      0.70233333 0.702     ]

Feature importances:  [0.19216551 0.47159158 0.33624291]


In [ ]:
# Selected features
features = X_train.columns[selector.get_support()]
print( features )
print( len(features) )

# Transform the training and test data after the RFE
X_train_rfe = selector.transform(X_train)
X_test_rfe = selector.transform(X_test)

In [ ]:
# Get the score of Gradient Boosting Model after the RFE
get_score_gradient_boosting(X_train_rfe, X_test_rfe, y_train, y_test)

## 4.2 Get best parameters of Gradien Boosting model with GridSearch and Cross-validation (GridSearchCV)

### Parameter space

In [ ]:
# Parameter space
param_grid = {
    "max_depth":[3,5],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "n_estimators":[10, 100]
    }

### GridSearchCV

In [ ]:
# GridsearchCV
clf = GridSearchCV(GradientBoostingClassifier(), param_grid=param_grid, cv=3, n_jobs=-1)
clf.fit(X_train_rfe, y_train)

# Get best params
print(clf.score(X_train_rfe, y_train))
print(clf.best_params_)

In [ ]:
# Re-train the model with best parameter from GridSearchCV
best_params = clf.best_params_
gbc = GradientBoostingClassifier(max_features=best_params['max_features'], n_estimators=best_params['n_estimators'],
                                 max_depth=best_params['max_depth'], criterion=best_params['criterion'], random_state=42)
gbc.fit(X_train_rfe, y_train)

In [ ]:
# Predict using test data
pred = gbc.predict(X_test_rfe)
# Get the score
print("Accuracy on test data: ", accuracy_score(y_test, pred))

In [ ]:
X_new = df_sub.drop(['DELAYED'], axis=1, inplace=False)[8000:8010]
X_new

In [ ]:
df_sub['DELAYED'][8000:8010]

In [ ]:
X_new_rfe = selector.transform(X_new)

print(X_new_rfe.shape)
X_new_rfe

In [ ]:
print('Prediction: ', gbc.predict(X_new_rfe).tolist())
print('True labels: ', df_sub['DELAYED'][8000:8010].tolist())

## 4.3 Get best parameters of Gradien Boosting model with Hyperopt

### Parameter space

In [ ]:
# param_space = {
#     'max_depth': hp.choice('max_depth', range(1,20)),
#     'max_features': hp.choice('max_features', range(1,150)),
#     'n_estimators': hp.choice('n_estimators', range(100,500)),
#     'criterion': hp.choice('criterion', ["gini", "entropy"])
#     }
# 'n_estimators': ho_scope.int(hp.quniform('n_estimators', low=100, high=300, q=25)), 
#             'criterion': hp.choice('criterion', ['gini', 'entropy']), 
#             'max_features': hp.uniform('max_features', low=0.25, high=0.75)    

# param_grid = {
#     "max_depth":[3,5],
#     "max_features":["log2","sqrt"],
#     "criterion": ["friedman_mse",  "mae"],
#     "n_estimators":[10, 100]
#     }

# hp_space_clf1 = {
#     'poly': {
#         'degree': 1 + hp.randint('degree', 1 + 1)
#     }, 
#     'clf': {
#         'n_estimators': ho_scope.int(hp.quniform('n_estimators', low=100, high=300, q=25)), 
#         'criterion': hp.choice('criterion', ['gini', 'entropy']),
#         'max_features': hp.choice('max_features', range(1,150)),
#         'max_depth': hp.choice('max_depth', range(1,20))
#     }
# }

In [ ]:
# Number of features in X_train
nfeatures = len( X_train.columns )
print('Number of features in X_train: ', nfeatures)

In [ ]:
# Params space
hp_space_clf = {
    'clf': {
        'n_estimators': ho_scope.int(hp.quniform('n_estimators', low=100, high=300, q=25)), 
        'criterion': hp.choice('criterion', ['friedman_mse', 'mse', 'mae']),
        'max_features': hp.choice('max_features', range(1,nfeatures)),
        'max_depth': hp.choice('max_depth', range(1,20))
    }
}

# Draw random sample to see if hyperspace is correctly defined
ho_sample(hp_space_clf)

### Defining model

In [ ]:
def f_clf(hps):
    """
    Constructs estimator
    
    Parameters:
    ----------------
    hps : sample point from search space
    
    Returns:
    ----------------
    model : sklearn.Pipeline.pipeline with hyperparameters set up as per hps
    """
    
    # Assembing pipeline if needed
    # model = Pipeline([
    #     ('clf', GradientBoostingClassifier(**hps['clf'], random_state=42))
    # ])
    
    model = GradientBoostingClassifier(**hps['clf'], random_state=42)
    
    return model

### Defining objective function

Define function to minimize. I'll use cross-validation score on train set.

In [ ]:
def fcn_to_minimize(hps, X, y, ncv=5):
    """
    Target function for optimization
    
    Parameters:
    ----------------
    hps : sample point from search space
    X : feature matrix
    y : target array
    ncv : number of folds for cross-validation
    
    Returns:
    ----------------
    : target function value (negative mean cross-val score)
    """
    
    model = f_clf(hps)

    # cv_res = cross_val_score(model, X, y, cv=StratifiedKFold(ncv, shuffle=True, random_state=SEED), 
    #                          scoring='roc_auc', n_jobs=-1)
    
    cv_res = cross_val_score(model, X, y, cv=StratifiedKFold(ncv, shuffle=True, random_state=31), scoring='f1_micro', n_jobs=-1)
    # cv_res = cross_val_score(model, X, y, cv=5, n_jobs=-1)
    # cv_res = cross_val_score(model, X, y, cv=StratifiedKFold(ncv, shuffle=True, random_state=42), n_jobs=-1)
    acc = cv_res.mean()
    
    print(cv_res)
    print("Accuracy: %0.2f (+/- %0.2f)" % (cv_res.mean(), cv_res.std() * 2))
    return {'loss': -acc, 'status': STATUS_OK}

### Running optimization

Run optimization for 100 rounds using TPE algorithm (the Tree-structured Parzen Estimator), meaning that we use TPE to suggest next sample values based on previous function evaluations. We'll use `Trials` class objects to keep track of optimization history.

**Note**: We're binding `X` and `y` arguments of target function to `X1_train` and `y1_train` respectively, using `functools.partial`, since target function of `fmin` may accept only a search space point.

In [ ]:
trials_clf = Trials()
best_clf = fmin(partial(fcn_to_minimize, X=X_train_rfe, y=y_train), 
                 hp_space_clf, algo=tpe.suggest, max_evals=50, 
                 trials=trials_clf)

In [ ]:
# Best parameters from Hyperopt
print(space_eval(hp_space_clf, best_clf))

print('Best parameters from Hyperopt: \n')
best_clf

# {'clf': {'criterion': 'mae', 'max_depth': 11, 'max_features': 7, 'n_estimators': 225}, 'poly': {'degree': 1}}
# {'criterion': 2,
#  'degree': 0,
#  'max_depth': 10,
#  'max_features': 6,
#  'n_estimators': 225.0}

### Model performance on test set

In [ ]:
# Building and fitting classifier with best parameters
clf = f_clf( space_eval(hp_space_clf, best_clf) ).fit(X_train_rfe, y_train)

# Calculating performance on test set
predictions = clf.predict(X_test_rfe)

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))

print("Classification Report:")
print(classification_report(y_test, predictions))

print('Best parameters:')
print(space_eval(hp_space_clf, best_clf))

In [ ]:
# Best trial
print('Score from Best trial: ', -trials_clf.best_trial['result']['loss'] )

In [ ]:
# Accuracy score
print('Accuracy score of the model on test set: ', accuracy_score(y_test, predictions)

In [ ]:
# Accuracy score using predict_proba
pred_prob_y_test = clf.predict_proba(X_test_rfe)
preds = np.argmax(pred_prob_y_test, axis=1)
accuracy_score(y_test, preds)

Test set accuracy and cross-validation scores are consistent.